In [2]:
const Float = Float64
"""
    Particle type

`nDim::Int`: dimension of the parameter space to be explored

`position::Array{Float, 1}`: current position of the particle

`velocity::Array{Float, 1}`: current velocity of the particle

`pBest::Array{Float, 1}`: the position at which the particle has 
the best-fit value through particle's history

`lBest::Array{Float, 1}`: the position at which the local group has 
the best-fit value through local group's history

`fitValue::Float`: the fit-value at current position

`fitpBest::Float`: the fit-value at `pBest`

`fitlBest::Float`: the fit-value at `lBest`

`nFitEval::Int`: number of the evaluation of the fitness function 
(which does not take the steps outside the parameter space into account)
"""
mutable struct Particle
    nDim::Int
    
    position::Array{Float, 1}
    velocity::Array{Float, 1}
    pBest::Array{Float, 1}
    lBest::Array{Float, 1}
    
    fitValue::Float
    fitpBest::Float
    fitlBest::Float
    
    nFitEval::Int
    
    # initialize the particle
    # with the lBest = pBest = position = random numbers
    # and fitlBest = fitpBest = fitValue = Inf
    # These setups will be updated by the initiation of the swarm
    function Particle(nDim::Int)
        position = rand(nDim)
        velocity = rand(nDim) - position
        pBest = position
        lBest = position
        
        fitValue = Inf
        fitpBest = fitValue
        fitlBest = fitValue
        
        nFitEval = 0
        
        new(nDim, position, velocity, pBest, lBest, 
            fitValue, fitpBest, fitlBest, nFitEval)
    end       
end

# test
# base_model after destructing have 185 features so nDim=185
# p1 = Particle(185)
# p1.fitValue

Particle

In [3]:
"""
    initFitValue!(fitFunc::Function, p::Particle)

Initiate the `fitValue` for the `p` particle using the fitness 
function `fitFunc`.
"""
function initFitValue!(fitFunc::Function, p::Particle)
#     p.fitValue = fitFunc(p.position)
#     p.fitValue = fitFunc(p.position)[1]
    θ1, re1 = Flux.destructure(base_model)
    p.position=θ1
    p.fitValue = fitFunc(p.position,re1)
    
    # update nFitEval
    p.nFitEval += 1
    nothing
end

# test
# x=rand(185)
# fitFunc(x) = (base_model(x))[1]
# fitFunc(θ,re)=mean(vec(re(θ)(X_train_std).>0.5).==y_train)
# p2=Particle(185)
# print(p2)
# initFitValue!(fitFunc,p2)

initFitValue!

In [4]:
"""
    updatePositionAndFitValue!(fitFunc::Function, nDim::Int, p::Particle)

Update the `position` and `fitValue` for the `p` particle using 
the fitness function `fitFunc` with `nDim` parameters.
"""
function updatePositionAndFitValue!(fitFunc::Function, p::Particle)
    p.position += p.velocity
    
    # if position is outside the paramter space, we set fitValue = Inf
    for x in p.position
        if (x < 0 || x > 1)
            p.fitValue = Inf
            return
        end
    end
    # update nFitEval
    p.nFitEval += 1
#     p.fitValue = fitFunc(p.position)
#     p.fitValue = fitFunc(p.position)[1]
    θ1, re1 = Flux.destructure(base_model)
    p.fitValue = fitFunc(p.position,re1)

    
    nothing
end

# test
# fitFunc(x) = x[1]^2 + x[2]^2
# p2 = Particle(185)
# println(p2)
# fitFunc(θ,re)=mean(vec(re(θ)(X_train_std).>0.5).==y_train)
# initFitValue!(fitFunc, p2)
# println(p2)
# updatePositionAndFitValue!(fitFunc, p2)
# println(p2)

updatePositionAndFitValue!

In [34]:

"""
    updatepBestAndFitpBest!(p::Particle)

Update the `pBest` and `fitpBest` for `p` particle.
"""
function updatepBestAndFitpBest!(p::Particle)
    if p.fitValue < p.fitpBest 
        p.fitpBest  = p.fitValue
        p.pBest = p.position
        θ1, re1 = Flux.destructure(base_model)
        base_model=re1(p.pBest)
    end
    nothing
end




updatepBestAndFitpBest!

In [6]:

"""
    updateVelocity!(p::Particle, w::Float, c1::Float=c1, c2::Float=c2)

Update the `velocity` of the particle.
"""
function updateVelocity!(p::Particle, w::Float, c1::Float, c2::Float)
    p.velocity = w * p.velocity + 
    c1 * rand() * (p.pBest - p.position) + 
    c2 * rand() * (p.lBest - p.position)
    
    nothing
end



updateVelocity!

In [7]:

"""
    neiborIndices(i::Int, nNeighbor::Int, nParticle::Int)

Return the indices for the `nNeighbor` neiborhoods of the `i`th particle
in a swarm with `nParticle` particles.
"""
function neiborIndices(i::Int, nNeibor::Int, nParticle::Int)
    
    # number of neighbors should be larger than 3
    nNeibor = max(3, nNeibor)
    
    # number of neighbors on the left side of i-th particle
    nLeft = (nNeibor - 1) ÷ 2
    
    # the index of the starting particle in the local group
    startIndex = (i - nLeft)
    
    # the index of the ending particle in the local group
    endIndex = startIndex + nNeibor -1
    
    # indices for the local group
    indices = collect(startIndex:endIndex)
    
    # ajust the indices to be in the range(1:nParticle)
    for i in 1:nNeibor
        if indices[i] < 1
            indices[i] += nParticle
        elseif indices[i] > nParticle
            indices[i] -= nParticle
        end
    end
    
    indices
end


neiborIndices

In [8]:


"""
    Swarm type

`fitFunc::Function`: fitness function to be evaluated

`nDim::Int`: dimension of the parameter space to be explored

`nParticle::Int`: number of particles in a swarm
    
`nNeibor::Int`: number of neighborhoods (particles) in a local group
    
`nInter::Int`: number of iterations for each particles to move forward

`c1::Float`: cognative constant

`c2::Float`: social constant

`wMax::Float`: the maximum value of inertia weight

`wMin::Float`: the minimum value of inertia weight

`w::Float`: the current value of inertia weight
    
`gBest::Array{Float, 1}`: the position at which the swarm has 
the best-fit value through the history
    
`fitgBest::Float`: the fit-value at `gBest`
    
`particles::Array{Particle, 1}`: the particles in a swarm

`nFitEvals::Int`: number of the evaluation of the fitness function 
(which does not take the steps outside the parameter space into account)
"""
mutable struct Swarm
    fitFunc::Function
    nDim::Int
    
    nParticle::Int
    nNeibor::Int
    nInter::Int
    
    c1::Float
    c2::Float
    
    wMax::Float
    wMin::Float
    w::Float
    
    gBest::Array{Float, 1}    
    fitgBest::Float
    
    particles::Array{Particle, 1}
    
    nFitEvals::Int
    
    # initialize the swarm
    function Swarm(fitFunc::Function, nDim::Int; 
            nParticle::Int=40, 
            nNeibor::Int=3, nInter::Int=2000,
            c1::Float=2.0, c2::Float=2.0,
            wMax::Float=0.9, wMin::Float=0.4)
        
        if nNeibor > nParticle
            error("Number of particles in a local group should not exceed 
                the totoal number of particles in the swarm!")
        end    
        
        w = wMax
        
        gBest = rand(nDim)
        fitgBest = Inf
        
        # initialize the swarm with nParticle
        particles = [Particle(nDim) for i in 1:nParticle]
        
        
        nFitEvals = 0
                
        new(fitFunc, nDim, nParticle, nNeibor, nInter, 
            c1, c2, wMax, wMin, w, gBest, 
            fitgBest, particles, nFitEvals)        
    end       
end



Swarm

In [9]:


"""
    updatelBestAndFitlBest!(s::Swarm)

Update lBest and fitlBest for each particle in the swarm `s`.
"""        
function updatelBestAndFitlBest!(s::Swarm)
    for i in 1:s.nParticle
        neiborIds = neiborIndices(i, s.nNeibor, s.nParticle)
        neiborFits = [s.particles[Id].fitValue for Id in neiborIds]
        fitlBest, index = findmin(neiborFits)
        
        if fitlBest < s.particles[i].fitlBest
            # neibor == local group
            lBest = s.particles[neiborIds[index]].position
            s.particles[i].lBest = lBest
            s.particles[i].fitlBest = fitlBest
        end
    end
    nothing
end

# test

updatelBestAndFitlBest!

In [10]:

"""
    updategBestAndFitgBest!(s::Swarm)

Update gBest and fitgBest for the swarm `s`.
"""        
function updategBestAndFitgBest!(s::Swarm)
    
    gFits = [particle.fitValue for particle in s.particles]
    fitgBest, index = findmin(gFits)
    if fitgBest < s.fitgBest
        s.gBest = s.particles[index].position   
        s.fitgBest = fitgBest
    end
    nothing
end


updategBestAndFitgBest!

In [11]:
"""
    initSwarm(s::Swarm)

Initiation (0st iteration) the swarm `s`.
"""
function initSwarm(s::Swarm)
    
    # initiate the fitValue for each particle
    for particle in s.particles
        initFitValue!(s.fitFunc, particle)
        updatepBestAndFitpBest!(particle)
    end
    
    # update lBest and fitlBest for the swarm
    updatelBestAndFitlBest!(s)
    
    # update gBest and fitgBest for the swarm
    updategBestAndFitgBest!(s)
    
    nothing
end


initSwarm

In [12]:
"""
    updateInertia!(s::Swarm)

Update the inertia weight after each iteration.
"""
function updateInertia!(s::Swarm)
    dw = (s.wMax - s.wMin)/s.nInter
    s.w -= dw
    
    nothing
end


updateInertia!

In [13]:
"""
    updateVelocity!(s::Swarm)

Update the `velocity` for each particle in the swarm `s`.
"""
function updateVelocity!(s::Swarm)
    for particle in s.particles
        updateVelocity!(particle, s.w, s.c1, s.c2)
    end        
    nothing
end


updateVelocity!

In [14]:

"""
    updatePositionAndFitValue!(s::Swarm)

Update the `position` and `fitValue` for each particle in the swarm `s`.
"""
function updatePositionAndFitValue!(s::Swarm)
    for particle in s.particles
        updatePositionAndFitValue!(s.fitFunc, particle)
    end        
    nothing
end


updatePositionAndFitValue!

In [15]:


"""
    updateSwarm(s::Swarm)

One iteration for the swarm `s`.
"""
function updateSwarm(s::Swarm)
    # update the velocity for each particle in the swarm
    updateVelocity!(s::Swarm)
    
    # update the position and fitValue for each particle in the swarm
    updatePositionAndFitValue!(s::Swarm)
    
    # update the lBest and fitlBest for each particle in the swarm
    updatelBestAndFitlBest!(s::Swarm)
    
    # update the gBest and fitgBest for the swarm
    updategBestAndFitgBest!(s::Swarm) 
    
    # update the inertia weigh w for each particle in the swarm
    updateInertia!(s::Swarm)
    
    nothing 
end


updateSwarm

In [16]:
using Plots
using LaTeXStrings

In [70]:
function onePsoRun(fitFunc::Function, nDim::Int; nParticle=100, nInter::Int=4000)
    s = Swarm(fitFunc, nDim, nParticle=nParticle, nInter=nInter)
    initSwarm(s)
    
    for i in s.nInter
        updateSwarm(s)
    end
    s.gBest
end
# fitFunc(x) = (base_model(x))
# accuracy(x, y) = mean(vec(base_model(x) .> 0.5) .== y)
fitFunc(θ,re)=mean(vec(re(θ)(X_train_std).>0.5).==y_train)
nDim = 185

nParticle = 100
nInter = 20
nRun = 10
xs = []
ys = zeros(nRun)

θ, re = Flux.destructure(base_model)

for i in 1:nRun
#     xs[i], ys[i] = onePsoRun(fitFunc, nDim=21, nParticle=nParticle, nInter=nInter)
    xs =  onePsoRun(fitFunc, 185, nParticle=nParticle, nInter=nInter)
    ys[i] =  mean(re(xs)(X_train_std))

end

In [77]:
ys

10-element Vector{Float64}:
 0.6797492052072394
 0.6004509116356114
 0.5727904704388198
 0.6834818873833022
 0.6882469836732197
 0.6090509095036966
 0.8128145632163694
 0.5166529030419913
 0.7076845130891805
 0.7581591906736167

In [78]:
mean(ys)

0.6629081537863046

In [20]:
using Evolutionary
using Flux
using Flux: onehot, onecold, logitcrossentropy #, throttle, @epochs
using MLDatasets
using Random
using StableRNGs

In [21]:
using ZipFile, CSV, DataFrames, Random, StatsBase , Plots, Statistics

# read file from zip archive

z = ZipFile.Reader("results.zip")

# identify the right file in zip

# The diabetes dataset I found through kaggle have done split into 2 classifiers and select 50 percents of result from each 
# label. So i think there is more work in my dataset

a_file_in_zip = filter(x->x.name == "diabetes_binary_5050split_health_indicators_BRFSS2015.csv", z.files)[1]

#avoid changing the original files in the zip file. However, the dataset will not change but whatever.

a_copy = CSV.File(a_file_in_zip) |> DataFrame

close(z)

#show the dataset

a_copy

,Diabetes_binary,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,0.0,1.0,1.0,25.0,1.0,0.0,0.0
2,0.0,0.0,0.0,0.0,25.0,1.0,0.0,0.0
3,0.0,1.0,1.0,1.0,29.0,1.0,0.0,0.0
4,0.0,0.0,0.0,1.0,26.0,1.0,0.0,0.0
5,0.0,0.0,0.0,1.0,29.0,1.0,0.0,0.0
6,0.0,0.0,0.0,0.0,24.0,0.0,0.0,0.0
7,0.0,1.0,1.0,1.0,36.0,0.0,0.0,0.0
8,0.0,0.0,0.0,1.0,26.0,1.0,0.0,0.0
9,0.0,0.0,0.0,1.0,21.0,0.0,0.0,0.0


In [22]:
# Transfer DataFrame to matrix form

df=a_copy|>Tables.matrix

# Transfer the dataset to 2-classifiers. df_0 represents the result is 0, df_1 represents the result is 1.
# Due to my datasset is binary problems, and each result is 50 percents of the whole dataset. So i didn't add any other pre-actions for dataset. 

df_0 = df[df[:,1] .== 0, :]
df_1 = df[df[:,1] .== 1, :]

35346×22 Matrix{Float64}:
 1.0  1.0  1.0  1.0  30.0  1.0  0.0  1.0  …  30.0  1.0  0.0   9.0  5.0  1.0
 1.0  0.0  0.0  1.0  25.0  1.0  0.0  0.0      0.0  0.0  1.0  13.0  6.0  8.0
 1.0  1.0  1.0  1.0  28.0  0.0  0.0  0.0      0.0  1.0  0.0  11.0  4.0  6.0
 1.0  0.0  0.0  1.0  23.0  1.0  0.0  0.0      0.0  0.0  1.0   7.0  5.0  6.0
 1.0  1.0  0.0  1.0  27.0  0.0  0.0  0.0      0.0  0.0  0.0  13.0  5.0  4.0
 1.0  1.0  1.0  1.0  37.0  1.0  1.0  1.0  …   0.0  1.0  1.0  10.0  6.0  5.0
 1.0  1.0  1.0  1.0  28.0  1.0  0.0  1.0      0.0  0.0  1.0  12.0  2.0  4.0
 1.0  1.0  1.0  1.0  27.0  1.0  0.0  0.0     20.0  1.0  0.0   8.0  4.0  7.0
 1.0  1.0  1.0  1.0  34.0  1.0  1.0  0.0      7.0  1.0  0.0   9.0  5.0  4.0
 1.0  1.0  1.0  1.0  24.0  1.0  0.0  0.0      0.0  0.0  0.0  12.0  3.0  3.0
 1.0  1.0  0.0  1.0  31.0  0.0  0.0  0.0  …   5.0  0.0  0.0  13.0  4.0  4.0
 1.0  1.0  1.0  1.0  33.0  1.0  0.0  0.0     30.0  1.0  0.0  11.0  4.0  2.0
 1.0  1.0  1.0  1.0  27.0  1.0  0.0  0.0     30.0  1.0  0.0  1

In [23]:
# The columns of matrix is 35346(here just using 35000 is the same), using random sub-sequence to select 70 percents of 
# columns as the train data.

sample = randsubseq(1:35000, 0.7)
train_df = vcat(df_0[sample, :], df_1[sample, :])

# Then from the not selected columns (which is 30 percents) to select the test data.

notsample = [i for i in 1:35000 if isempty(searchsorted(sample, i))]
test_df = vcat(df_0[notsample, :], df_1[notsample, :])

21136×22 Matrix{Float64}:
 0.0  0.0  1.0  1.0  25.0  1.0  0.0  0.0  …   2.0  0.0  0.0  11.0  5.0  7.0
 0.0  1.0  1.0  1.0  29.0  1.0  0.0  0.0      1.0  0.0  1.0  13.0  5.0  8.0
 0.0  0.0  0.0  1.0  29.0  1.0  0.0  0.0      0.0  0.0  1.0   4.0  4.0  4.0
 0.0  0.0  0.0  1.0  26.0  0.0  0.0  0.0      0.0  0.0  1.0   2.0  5.0  5.0
 0.0  0.0  0.0  1.0  31.0  0.0  0.0  0.0      2.0  0.0  0.0   3.0  6.0  8.0
 0.0  0.0  0.0  1.0  23.0  0.0  0.0  0.0  …   3.0  0.0  0.0   2.0  5.0  8.0
 0.0  1.0  1.0  1.0  29.0  0.0  0.0  0.0      0.0  0.0  0.0  12.0  3.0  2.0
 0.0  0.0  0.0  1.0  25.0  0.0  0.0  0.0      3.0  0.0  1.0   4.0  6.0  8.0
 0.0  0.0  1.0  1.0  30.0  1.0  0.0  0.0      1.0  1.0  0.0   8.0  5.0  8.0
 0.0  0.0  0.0  1.0  27.0  0.0  0.0  0.0      1.0  0.0  0.0   6.0  6.0  7.0
 0.0  0.0  0.0  1.0  26.0  1.0  0.0  0.0  …   0.0  0.0  0.0   5.0  6.0  8.0
 0.0  1.0  1.0  1.0  25.0  0.0  1.0  1.0     14.0  0.0  1.0   8.0  5.0  2.0
 0.0  1.0  0.0  1.0  22.0  1.0  0.0  0.0      0.0  0.0  0.0  1

In [24]:
# Divide the columns into features and result. From my dataset, From 2 to 22 columns are the attributes of whether diabetes or not.

X_train = train_df[:, 2:22]
X_test = test_df[:, 2:22]

y_train = train_df[:, 1]
y_test = test_df[:, 1]

21136-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0

In [25]:
using Distributions

# Transfer the X_train to 1 diminension.

dt = fit(ZScoreTransform, X_train, dims=1)

# Using StateBase package to transfer X_train and X_test(make them standard) to the same formate as dt.

X_train_std = StatsBase.transform(dt, X_train)
X_test_std = StatsBase.transform(dt, X_test)

# Transpose the X_train_std and X_test_std.
    
X_train_std, X_test_std= transpose(X_train_std), transpose(X_test_std)

([-1.1410765551724502 -1.1410765551724502 … 0.876347454956597 0.876347454956597; -1.050114350078777 -1.050114350078777 … 0.9522577564633818 -1.050114350078777; … ; 1.0478129065438462 1.0478129065438462 … -0.8998563670927371 -0.8998563670927371; 1.0581893275975054 1.0581893275975054 … -2.1702923971605177 -0.786657372264222], [-1.1410765551724502 0.876347454956597 … 0.876347454956597 0.876347454956597; 0.9522577564633818 0.9522577564633818 … -1.050114350078777 -1.050114350078777; … ; 0.07397826972555453 0.07397826972555453 … 1.0478129065438462 -1.8736910039110288; 0.5969776526320735 1.0581893275975054 … -1.7090807221950857 -0.786657372264222])

In [32]:
base_model=Chain(Dense(21, 8, sigmoid), Dense(8, 1, sigmoid))

Chain(
  Dense(21, 8, σ),                      # 176 parameters
  Dense(8, 1, σ),                       # 9 parameters
)                   # Total: 4 arrays, 185 parameters, 996 bytes.

In [27]:
using Flux: logitbinarycrossentropy
opt = ADAM()
loss(x, y) = logitbinarycrossentropy(vec(base_model(x)), y)
accuracy(x, y) = mean(vec(base_model(x) .> 0.5) .== y)
evalcb1() = @show(loss(X_test_std, y_test))
throttled_cb = Flux.throttle(evalcb1, 0.01)
for i in 1:500
    Flux.train!(loss, Flux.params(base_model), [(X_train_std, y_train)], opt, cb = throttled_cb)
end

loss(X_test_std, y_test) = 0.7577324821272544
loss(X_test_std, y_test) = 0.7563078757316315
loss(X_test_std, y_test) = 0.7548730795304073
loss(X_test_std, y_test) = 0.7534286971497473
loss(X_test_std, y_test) = 0.751975449105833
loss(X_test_std, y_test) = 0.7505141822402857
loss(X_test_std, y_test) = 0.7490458710867265
loss(X_test_std, y_test) = 0.7475716190970703
loss(X_test_std, y_test) = 0.746092647873391
loss(X_test_std, y_test) = 0.7446102859003362
loss(X_test_std, y_test) = 0.7431259623459242
loss(X_test_std, y_test) = 0.7416411899685292
loss(X_test_std, y_test) = 0.7401575478794662
loss(X_test_std, y_test) = 0.7386766555273439
loss(X_test_std, y_test) = 0.7372001697275763
loss(X_test_std, y_test) = 0.7357297509451778
loss(X_test_std, y_test) = 0.7342670677144811
loss(X_test_std, y_test) = 0.7328137615960615
loss(X_test_std, y_test) = 0.7313714416861861
loss(X_test_std, y_test) = 0.7299416636030704
loss(X_test_std, y_test) = 0.7285259218852341
loss(X_test_std, y_test) = 0.7271256

In [28]:
@info "MLP" loss=loss(X_train_std,y_train) accuracy = accuracy(X_train_std,y_train)

┌ Info: MLP
│   loss = 0.657639259564815
│   accuracy = 0.668897347740668
└ @ Main In[28]:1


In [88]:
# Plots.plot(ys,label=["train_acc"])
# 

In [86]:
"""
    In order to compute for confusion matrix
    Just from hw1
"""
#TT
TT=sum(vec(base_model(X_test_std) .> 0.5) .== 1)
#TF
TF=sum(vec(base_model(X_test_std) .> 0.5) .== 0)
#FT
FT=sum(vec(base_model(X_test_std) .< 0.5) .== 1)
#FF
FF=sum(vec(base_model(X_test_std) .< 0.5) .== 0)
println("confusion matrix of test data in best model:")
println("true_true_result->$TT,  true_false_result->$TF")
println("false_false_result->$FF,   false_true_result->$FT")

confusion matrix of test data in best model:
true_true_result->12839,  true_false_result->7306
false_false_result->11728,   false_true_result->6174
